In [1]:
#package installs
#E:\Python310\python.exe -m pip install --upgrade pip
#!E:\Python310\python.exe -m pip install git+https://github.com/openai/whisper.git soundfile
#!E:\Python310\Scripts\pip3.exe install ffmpeg av
#!E:\Python310\python.exe -m pip uninstall torch torchvision torchaudio -y
#!E:\Python310\python.exe -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [2]:
#Global variables
import socket
import os
ENV_HOSTNAME = socket.gethostname()
print('ENV_HOSTNAME:' + ENV_HOSTNAME)

#store defaults for Jacob here:
ENV_FOLDER_DATA = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data\\'
ENV_FOLDER_DATA_PROC = 'C:\\Users\\jakes\\Documents\\COMS 4732 - Computer Vision\\W4732CompVisFinal\\Data_Proc\\'
ENV_PRATT = ''


if ENV_HOSTNAME == 'JWGamingPC':
    ENV_FOLDER_DATA = 'E:\\W4732 Computer Vision\\Final Paper Data\\'
    ENV_FOLDER_DATA_PROC = 'E:\\W4732 Computer Vision\\Final Paper Data Proc\\'
    ENV_PRATT = ''

print('ENV_FOLDER_DATA:' + ENV_FOLDER_DATA)
print('ENV_PRATT:' + ENV_PRATT)

# Create folder structure
import os
os.makedirs(ENV_FOLDER_DATA_PROC + 'segmentation', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'targetdf', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'pratt', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'eps', exist_ok=True)
os.makedirs(ENV_FOLDER_DATA_PROC + 'clips', exist_ok=True)


ENV_HOSTNAME:JWGamingPC
ENV_FOLDER_DATA:E:\W4732 Computer Vision\Final Paper Data\
ENV_PRATT:


In [3]:
#create model result
#PATH_MODEL = ENV_FOLDER_DATA_PROC + 'charts\\' + 'pitchVidNetV3.pt'

import torch
import torch.nn.functional as F

#torch.save(a.state_dict(), ENV_FOLDER_DATA_PROC + 'charts\\' + "JWPitchModv04Fin.pt")
class JWVideoConvNetV4Fin(torch.nn.Module):
    def __init__(self, in_channels=1, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        
        
        self.conv1 = torch.nn.Conv3d(in_channels=in_channels, out_channels=4, kernel_size=15, padding=3, stride=5)
        
        self.lin1 = torch.nn.Linear(26460, 4096)
        self.lin2 = torch.nn.Linear(4096, 1024)
        self.lin3 = torch.nn.Linear(1024, num_classes)

        self.drop = torch.nn.Dropout(0.1)
        self.sigmoid = torch.nn.Sigmoid()
        self.maxpool = torch.nn.MaxPool3d(kernel_size=4, stride=3)

    def forward(self, x):
        # TODO: Implement the forward pass with using the layers defined above
        #       and the proper activation functions
        #print("input: ", x.shape)
        x = self.conv1(x)
        x = self.sigmoid(x)
        x = self.maxpool(x)

        x = torch.flatten(x,1)
        x = self.lin1(x)
        x = self.sigmoid(x)
        x = self.drop(x)

        x = self.lin2(x)
        x = self.sigmoid(x)
        x = self.drop(x)
        x = self.lin3(x)
        #print("fc3: ", x.shape)
        return x


class JWVideoConvNet(torch.nn.Module):
    def __init__(self, in_channels=1, num_classes=41):
        super().__init__()
        # TODO: Initialize network layers
        
        
        self.lin1 = torch.nn.Linear(271557, 4096,bias=False)
        self.lin2 = torch.nn.Linear(4096, 1024,bias=False)
        self.lin3 = torch.nn.Linear(1024, num_classes,bias=False)

        self.drop = torch.nn.Dropout(0.1)
        self.sigmoid = torch.nn.Sigmoid()
        self.maxpool = torch.nn.MaxPool3d(kernel_size=10, stride=3)

    def forward(self, x):
        # TODO: Implement the forward pass with using the layers defined above
        #       and the proper activation functions
        #print("input: ", x.shape)
        x = self.maxpool(x)

        x = torch.flatten(x,1)
        x = self.lin1(x)
        x = self.sigmoid(x)
        x = self.drop(x)

        x = self.lin2(x)
        x = self.sigmoid(x)
        x = self.drop(x)
        x = self.lin3(x)
        #print("fc3: ", x.shape)
        return x

a = JWVideoConvNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#a.load_state_dict(torch.load(ENV_FOLDER_DATA_PROC + 'charts\\' + 'JWPitchModv01.pt'))
a.to(device=device)

JWVideoConvNet(
  (lin1): Linear(in_features=271557, out_features=4096, bias=True)
  (lin2): Linear(in_features=4096, out_features=1024, bias=True)
  (lin3): Linear(in_features=1024, out_features=41, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
  (sigmoid): Sigmoid()
  (maxpool): MaxPool3d(kernel_size=10, stride=3, padding=0, dilation=1, ceil_mode=False)
)

In [4]:
## Model Training 1 ##
import pandas as pd
import json
import copy
import csv
import pickle
import torchvision
import itertools
import torch
import glob
import random



#get number after # but before space afterwards

#generate filepaths
folderpath_clipinfo = ENV_FOLDER_DATA_PROC + 'clipinfo\\'
os.makedirs(folderpath_clipinfo, exist_ok=True)
folderpath_sourcedf = ENV_FOLDER_DATA_PROC + 'sourcedf\\'
os.makedirs(folderpath_sourcedf, exist_ok=True)
folderpath_targetdf = ENV_FOLDER_DATA_PROC + 'targetdf\\'
folderpath_pratt = ENV_FOLDER_DATA_PROC + 'pratt\\' 
filepath_speakers = ENV_FOLDER_DATA_PROC + 'speakers.json'

## get overlap between sourcedf files and targetdf files
list_eps = []

#note - source is actually a dict of dict of tensors
dict_sourcedf = {}
list_sourcedf_eps = []
for path_pickle in glob.glob(folderpath_sourcedf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_sourcedf_eps.append(int_epnum)
    dict_sourcedf[int_epnum] = dict_temp

#note - target is actually a df
dict_targetdf = {}
list_targetdf_eps = []
for path_pickle in glob.glob(folderpath_targetdf + '*.pickle'):
    dict_temp = {}
    dict_temp['path'] = path_pickle
    str_basenm = os.path.basename(path_pickle)
    dict_temp['basenm'] = str_basenm
    dict_temp['str_epnum'] = str_basenm.split('.')[0]
    int_epnum = int(dict_temp['str_epnum'])
    dict_temp['int_epnum'] = int_epnum
    list_targetdf_eps.append(int_epnum)
    dict_targetdf[int_epnum] = dict_temp

#populate matches
for src_ep in list_sourcedf_eps:
    if src_ep in list_targetdf_eps:
        list_eps.append(src_ep)

#40 data elements
list_datacols = []
list_datacols.append('m')
for i in range(40):
    list_datacols.append('d' + str(i))

#define model pieces
NUM_EPOCHS = 100
TRAINING_TYPE = 'p'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#transform=torchvision.transforms.Normalize( mean=(255*0.5), std=(255*0.5) )
optimizer = torch.optim.Adam(a.parameters(), lr=0.001, betas=(0.5, 0.999))
criterion = torch.nn.MSELoss().to(device)


##Epoch Loop
for e in range(NUM_EPOCHS):
    print('Epoch:' + str(e))
    #make a copy of the episode list to pop out 2 at a time
    list_epocheps = copy.deepcopy(list_eps)
    random.shuffle(list_epocheps)

    list_loss = []

    while len(list_epocheps) > 0:
        #load two or 3 episodes into memory at a time
        list_tempeps = []
        for i in range(3):
            if len(list_epocheps) > 0:
                list_tempeps.append(list_epocheps.pop())
        #load pratt files into memory
        dict_target = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_targetdf[tempeps]['path'], 'rb') as file:
                dict_target[tempeps] =  pickle.load(file)
                    
        #load video files into memory
        dict_source = {}
        for i,tempeps in enumerate(list_tempeps):
            with open(dict_sourcedf[tempeps]['path'], 'rb') as file:
                dict_temp = pickle.load(file)
                dict_source[tempeps] = dict_temp

        #create a list of eps / clips and shuffle
        listtup_epseg = []
        for key,val in dict_source.items():
            for key2,val2 in val.items():
                tup_epseg = [key,key2]
                listtup_epseg.append( tup_epseg)
        random.shuffle(listtup_epseg)

        #loop for data elements
        for tup_epseg in listtup_epseg:
            int_epnum = tup_epseg[0]
            int_segnum = tup_epseg[1]
            #feed values into the predict / loss / optimize cycle
            inputs = None
            labels = None
            if int_epnum not in dict_source.keys():
                continue
            if int_segnum not in dict_source[int_epnum].keys():
                continue
            inputs = dict_source[int_epnum][int_segnum]
            #pad inputs
            for x in range(40 - len(inputs)):
                inputs.append( torch.empty((1,360,640), dtype=torch.int8) )
            inputs = torch.stack(inputs[:40], 0)
            #transform inputs
            #print(inputs.size())
            #torch.Size([40, 1, 360, 640])
            inputs = inputs.to(torch.float).movedim(0,1).unsqueeze(0) / 255.0
            #print(inputs.size())
            

            #check if labels exist
            label_df = dict_target[int_epnum].query('seg==' + str(int_segnum) + ' & type=="' + TRAINING_TYPE + '"' )
            if len(label_df) == 0:
                #print('No labels for ep:' + str(int_epnum) + ' seg:' + str(int_segnum))
                continue
            labels = torch.squeeze(torch.tensor(label_df[list_datacols].fillna(0).values).to(torch.float))
            #normalize labels 
            if TRAINING_TYPE == 'p':
                labels = labels / 600.0
            elif TRAINING_TYPE == 'i':
                labels = labels / 100.0
            elif TRAINING_TYPE == 's':
                labels = labels / 2
            elif TRAINING_TYPE == 'j':
                labels = labels
            elif TRAINING_TYPE == 'h':
                labels = (labels + 40) / 100
            

            inputs, labels = inputs.to(device), labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = a(inputs)
            loss = criterion(outputs.squeeze(), labels.squeeze())
            list_loss.append(loss)
            loss.backward()
            optimizer.step()

        # print statistics
        print(f'Epoch={e + 1} Loss={sum(list_loss):.3f}')




    #define a function that, given a targetdf value, look up the source - if both exists, use the data to train the model

#end loop through data
#33 minutes per epoch

Epoch:0
Epoch=1 Loss=44.614
Epoch=1 Loss=75.703
Epoch=1 Loss=106.711
Epoch=1 Loss=141.172
Epoch=1 Loss=164.622
Epoch=1 Loss=198.704
Epoch=1 Loss=231.393
Epoch=1 Loss=266.748
Epoch=1 Loss=298.775
Epoch=1 Loss=354.346
Epoch=1 Loss=396.106
Epoch=1 Loss=433.491
Epoch=1 Loss=464.146
Epoch=1 Loss=497.821
Epoch=1 Loss=548.629
Epoch:1
Epoch=2 Loss=33.794
Epoch=2 Loss=66.701
Epoch=2 Loss=101.108
Epoch=2 Loss=129.568
Epoch=2 Loss=160.242
Epoch=2 Loss=204.532
Epoch=2 Loss=252.494
Epoch=2 Loss=281.506
Epoch=2 Loss=317.388
Epoch=2 Loss=355.989
Epoch=2 Loss=389.933
Epoch=2 Loss=418.175
Epoch=2 Loss=438.869
Epoch=2 Loss=502.936
Epoch=2 Loss=521.826
Epoch:2
Epoch=3 Loss=24.062
Epoch=3 Loss=58.633
Epoch=3 Loss=87.570
Epoch=3 Loss=135.284
Epoch=3 Loss=185.820
Epoch=3 Loss=222.009
Epoch=3 Loss=285.287
Epoch=3 Loss=316.179
Epoch=3 Loss=351.149
Epoch=3 Loss=382.062
Epoch=3 Loss=417.831
Epoch=3 Loss=450.903
Epoch=3 Loss=477.690
Epoch=3 Loss=508.845
Epoch=3 Loss=526.407
Epoch:3
Epoch=4 Loss=30.614
Epoch=4 Lo

KeyboardInterrupt: 

In [5]:
#save model down for future use / training
torch.save(a.state_dict(), ENV_FOLDER_DATA_PROC + 'charts\\' + "JWPitchModv04Fin.pt")

In [ ]:
#have some metric to show 